# Hugging Face 感情分析クイックスタート

Google Colabで無料GPUを使って、AIモデルの学習を体験しましょう。

**所要時間**: 約5分

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/)

## 1. 環境設定

まず、GPUが有効になっているか確認します。

**設定方法**: メニュー → ランタイム → ランタイムのタイプを変更 → GPU を選択

In [ ]:
# GPU確認
import torch
print(f"GPU利用可能: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU名: {torch.cuda.get_device_name(0)}")

## 2. ライブラリのインストール

In [ ]:
!pip install -q transformers datasets accelerate evaluate

## 3. ライブラリのインポート

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    set_seed
)
from datasets import load_dataset
import evaluate
import numpy as np

# 再現性のためシード固定
set_seed(42)

# デバイス確認
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用デバイス: {device}")

## 4. データセットの読み込み

IMDb映画レビューデータセット（英語）を使用します。

In [ ]:
# データセット読み込み（少量版）
# データセット全体をロードしてシャッフル
full_dataset = load_dataset("imdb")
shuffled_train = full_dataset["train"].shuffle(seed=42)
shuffled_test = full_dataset["test"].shuffle(seed=42)

# バランスの取れたデータセットを作成
dataset = {
    "train": shuffled_train.select(range(1000)),
    "test": shuffled_test.select(range(200)),
}

print(f"訓練データ: {len(dataset['train'])}件")
print(f"テストデータ: {len(dataset['test'])}件")

# サンプル表示
print(f"\n例: {dataset['train'][0]['text'][:100]}...")
print(f"ラベル: {'ポジティブ' if dataset['train'][0]['label'] == 1 else 'ネガティブ'}")

## 5. モデルとトークナイザーの準備

In [ ]:
# モデル名（軽量版BERT）
model_name = "distilbert-base-uncased"

# トークナイザーとモデルの読み込み
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)
model.to(device)

print(f"モデル: {model_name}")
print(f"パラメータ数: {model.num_parameters():,}")

## 6. データの前処理

In [ ]:
def preprocess_function(examples):
    return tokenizer(
        examples['text'],
        padding="max_length",
        truncation=True,
        max_length=256
    )

# 前処理適用
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=['text']
)

print("前処理完了")

## 7. 評価関数の定義

In [ ]:
# 評価メトリクス
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

## 8. 学習の設定と実行

In [ ]:
# 学習設定
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    load_best_model_at_end=True,
    fp16=torch.cuda.is_available(),  # GPU時は混合精度
    report_to="none",  # wandbなどのログを無効化
)

# Trainer初期化
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    compute_metrics=compute_metrics,
)

print("学習を開始します...")
trainer.train()
print("学習完了!")

## 9. 評価結果の確認

In [ ]:
# 評価
eval_result = trainer.evaluate()

print("=" * 40)
print("最終結果")
print("=" * 40)
print(f"正解率: {eval_result['eval_accuracy']:.2%}")
print(f"損失: {eval_result['eval_loss']:.4f}")

## 10. 実際に予測してみよう

In [ ]:
# テストテキスト
test_texts = [
    "This movie was absolutely fantastic! I loved it.",
    "Terrible movie. Complete waste of time.",
    "It was okay, nothing special."
]

model.eval()
for text in test_texts:
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=256)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=-1)
    
    pred = "ポジティブ" if probs[0][1] > probs[0][0] else "ネガティブ"
    conf = max(probs[0]).item()
    
    print(f"入力: {text}")
    print(f"予測: {pred} (確信度: {conf:.1%})")
    print("-" * 50)

## 次のステップ

1. **データ量を増やす**: `train[:5000]` に変更
2. **エポック数を増やす**: `num_train_epochs=5`
3. **日本語モデルを試す**: `cl-tohoku/bert-base-japanese-v3`
4. **モデルを保存**: `trainer.save_model("my-model")`